In [1]:
from valentine import valentine_match
from valentine.algorithms import Coma
import pandas as pd

source = pd.read_csv("datasets/dblp_acm/source.csv")
target = pd.read_csv("datasets/dblp_acm/target.csv")
gt = pd.read_csv("datasets/dblp_acm/groundtruth.csv")

matches = valentine_match(source, target, Coma())
matches

{(('table_1', 'authors'), ('table_2', 'authors')): 0.9558824,
 (('table_1', 'title'), ('table_2', 'title')): 0.95,
 (('table_1', 'venue'), ('table_2', 'venue')): 0.95,
 (('table_1', 'year'), ('table_2', 'year')): 0.94642854,
 (('table_1', 'id'), ('table_2', 'id')): 0.9375}

In [10]:
from valentine.algorithms.matcher_results import MatcherResults
import pandas as pd

matches = pd.DataFrame({
    "source": ["authors", "title"],
    "target": ["authors", "title"],
})
valentine_dict = {}
for row in matches.itertuples():
    valentine_dict[(("source", row.source), ("target", row.target))] = 0
MatcherResults(valentine_dict)

{(('source', 'authors'), ('target', 'authors')): 0,
 (('source', 'title'), ('target', 'title')): 0}

In [2]:
from valentine.algorithms.matcher_results import MatcherResults
from matching import matching
from config import Config
from bdikit import BaseSchemaMatcher, match_schema, SchemaMatchers





config = Config("datasets/dblp_acm/source.csv", "datasets/dblp_acm/target.csv", "datasets/dblp_acm/groundtruth.csv")

ground_truth = config.get_ground_truth().sort_values(by="source")
source = config.get_source()[ground_truth["source"].values]
target = config.get_target()

matcher_instance = SchemaMatchers.get_instance("coma")
matches = matcher_instance.map(source, target)

# matches = match_schema(source, target, "coma")
matches

# valentine_set = set()
# for row in matches.itertuples():
#     valentine_set.add()
# MatcherResults(matches)

{'authors': 'authors',
 'title': 'title',
 'venue': 'venue',
 'year': 'year',
 'id': ''}

In [4]:

gt_set = set()
for row in gt.itertuples():
    gt_set.add((row.source, row.target))

gt_set

{('authors', 'authors'),
 ('id', 'id'),
 ('title', 'title'),
 ('venue', 'venue'),
 ('year', 'year')}

In [9]:
matches.get_metrics(gt_set)

{'Precision': 1.0,
 'F1Score': 0.888888888888889,
 'Recall': 0.8,
 'PrecisionTop10Percent': 1.0,
 'RecallAtSizeofGroundTruth': 1.0}

In [1]:
import os
import importlib.util
import sys

MATCHERS_PATH = "./matchers"

dirs_scan = [f.name for f in os.scandir(MATCHERS_PATH) if f.is_dir() and not f.name.startswith(".")]
for matcher_name in dirs_scan:
    sys.path.append(os.path.join(MATCHERS_PATH, matcher_name))
    spec = importlib.util.spec_from_file_location(f"matchers.{matcher_name}", os.path.join(MATCHERS_PATH, f"{matcher_name}/matching.py"))
    matcher = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(matcher)
    print(matcher.matching("dblp_acm", "datasets/dblp_acm"))

None


2024-10-03 01:10:17 INFO Using CPU.
2024-10-03 01:10:17 INFO begin to prepare the data ...
2024-10-03 01:10:18 INFO src effective table width: 5
2024-10-03 01:10:18 INFO tgt effective table width: 5
100%|██████████| 5/5 [01:03<00:00, 12.76s/it]
2024-10-03 01:11:28 INFO begin the eval-to-match process
100%|██████████| 1/1 [00:06<00:00,  6.90s/it]

{(('source', 'id'), ('target', 'id')): 0.9834933876991272, (('source', 'authors'), ('target', 'authors')): 0.9817944169044495, (('source', 'year'), ('target', 'year')): 0.9527695178985596, (('source', 'title'), ('target', 'title')): 0.935319185256958, (('source', 'venue'), ('target', 'venue')): 0.8851597905158997, (('source', 'title'), ('target', 'venue')): 0.85224449634552, (('source', 'venue'), ('target', 'title')): 0.7501089572906494, (('source', 'id'), ('target', 'year')): 0.4944625496864319, (('source', 'year'), ('target', 'id')): 0.4941103756427765, (('source', 'venue'), ('target', 'authors')): 0.3156331479549408, (('source', 'authors'), ('target', 'venue')): 0.28601881861686707, (('source', 'title'), ('target', 'authors')): 0.21937759220600128, (('source', 'authors'), ('target', 'title')): 0.1759164184331894, (('source', 'title'), ('target', 'year')): 0.13378700613975525, (('source', 'venue'), ('target', 'year')): 0.1022358387708664, (('source', 'authors'), ('target', 'id')): 0.

In [3]:
from bdikit.mapping_algorithms.column_mapping.topk_matchers import (
    CLTopkColumnMatcher,
)
from bdikit.models.contrastive_learning.cl_api import (
    DEFAULT_CL_MODEL,
)

api = CLTopkColumnMatcher(DEFAULT_CL_MODEL)

matches = api.get_recommendations(
            source, target, 20
        )

output = {}
for match in matches:
    source = match["source_column"]
    for candidate in match["top_k_columns"]:
        candidate_name = candidate.column_name
        candidate_score = candidate.score
        output[(("source", source), ("target", candidate_name))] = candidate_score
output

Extracting features from 5 columns...


  0%|          | 0/5 [00:00<?, ?it/s]

Extracting features from 5 columns...


  0%|          | 0/5 [00:00<?, ?it/s]

{(('source', 'id'), ('target', 'id')): 0.8343361,
 (('source', 'id'), ('target', 'year')): 0.40740085,
 (('source', 'id'), ('target', 'authors')): 0.3164454,
 (('source', 'id'), ('target', 'venue')): 0.054573573,
 (('source', 'id'), ('target', 'title')): -0.06622894,
 (('source', 'title'), ('target', 'title')): 0.5425582,
 (('source', 'title'), ('target', 'venue')): 0.34603542,
 (('source', 'title'), ('target', 'authors')): 0.10368133,
 (('source', 'title'), ('target', 'year')): 0.029006127,
 (('source', 'title'), ('target', 'id')): -0.099028006,
 (('source', 'authors'), ('target', 'authors')): 0.79599035,
 (('source', 'authors'), ('target', 'year')): 0.33636358,
 (('source', 'authors'), ('target', 'id')): 0.3267522,
 (('source', 'authors'), ('target', 'title')): 0.22946453,
 (('source', 'authors'), ('target', 'venue')): 0.18146417,
 (('source', 'venue'), ('target', 'venue')): 0.74062765,
 (('source', 'venue'), ('target', 'title')): 0.21765012,
 (('source', 'venue'), ('target', 'year')

In [5]:
from valentine.algorithms.matcher_results import MatcherResults

results = MatcherResults(output)

results.get_metrics(gt_set)

{'Precision': 1.0,
 'F1Score': 1.0,
 'Recall': 1.0,
 'PrecisionTop10Percent': 1.0,
 'RecallAtSizeofGroundTruth': 1.0}